In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline
%load_ext autoreload
%autoreload 2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np
import torch
import json
from pathlib import Path
from datetime import timedelta

In [3]:
# Check Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if (torch.cuda.is_available()):
    print(torch.cuda.get_device_name(0))

Using device: cuda
Tesla T4


In [4]:
# Load Config
# Ignoring config for now

In [5]:
# data_path = config['data']['path']
# print(data_path)

In [6]:
# Load operation ids with timestamps
df = pd.read_feather('../../data/minnie_splitlog_240927/minnie_splitlog_240927.feather')

In [7]:
df.head(5)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
0,2020-07-29 09:29:47.046010+00:00,1.596015e+09,117787.0,100298.500000,18901.000000,47222
1,2020-08-17 08:09:30.213000+00:00,1.597652e+09,122348.5,95767.500000,21351.000000,105222
2,2020-08-20 14:21:23.740000+00:00,1.597933e+09,122647.5,95861.000000,21358.000000,107815
3,2020-09-02 20:09:30.303000+00:00,1.599077e+09,162144.2,46332.355556,26524.933333,115686
4,2020-09-02 20:09:50.511000+00:00,1.599077e+09,159585.5,53807.000000,27261.000000,115687


In [8]:
df.tail(5)

,date,timestamp,mean_coord_x,mean_coord_y,mean_coord_z,operation_id
605765,2024-09-28 02:35:21.204000+00:00,1.727491e+09,116073.277778,59638.833333,17451.111111,1194265
605766,2024-09-28 02:35:41.694000+00:00,1.727491e+09,104502.821429,63632.113095,15037.690476,1194266
605767,2024-09-28 02:35:55.615000+00:00,1.727491e+09,119011.416667,75638.122024,24492.101190,1194267
605768,2024-09-28 02:36:46.009000+00:00,1.727491e+09,121120.620301,57906.807018,16755.139098,1194269
605769,2024-09-28 02:37:20.040000+00:00,1.727491e+09,104198.325000,63961.841667,15032.516667,1194270


In [9]:
# Load edit history
# [operation_id, sink supervoxel id, source_supervoxel id]
edits = np.load('../../data/minnie_splitlog_240927/minnie_splitlog_240927.npy')

In [10]:
edits.shape

(183743156, 3)

In [11]:
print(edits[0:10])

[[            47222 97197996596845829 97197996596845891]
 [            47222 97197996596845829 97197996596851211]
 [            47222 97197996596845829 97197996596845891]
 [            47222 97197996596845829 97197996596851211]
 [           105222 98393028632722115 98393028632725107]
 [           105222 98393028632725065 98393028632726303]
 [           105222 98393028632725065 98393028632734564]
 [           105222 98393028632725107 98393028632726303]
 [           105222 98393028632725107 98393028632728301]
 [           105222 98393028632726303 98393028632727324]]


In [12]:
print(edits[17000000])

[            304308 103111169795278386 103111169795297864]


In [13]:
print(edits[-10:])

[[          1194270 93458694519564700 93458694519567553]
 [          1194270 93388325775409047 93388325775413568]
 [          1194270 93388325775409047 93388325775410020]
 [          1194270 93388325775409047 93458694519564690]
 [          1194270 93458694519564690 93388325775409047]
 [          1194270 93458694519564690 93458694519564700]
 [          1194270 93458694519564690 93388325775409047]
 [          1194270 93458694519564690 93458694519564700]
 [          1194270 93388325775409047 93388325775410020]
 [          1194270 93388325775409047 93458694519564690]]


In [14]:
# To create a new token
# client = caveclient.CAVEclient()
# client.auth.setup_token(make_new=True)

In [15]:
# Initialize CAVE client
datastack_name = "minnie65_phase3_v1"
my_token = "64df9664ce8a28852ee99167c26a9e8d"
client = CAVEclient(datastack_name=datastack_name, auth_token=my_token)

In [16]:
# print the root ids at the 943 materialization, pre-edit and at edit time for the sink and source sv
def print_root_ids(df, operation_id, edits, materialization, client):
    # get source, sink svs for operation id
    indices = np.where(edits[:, 0] == operation_id)
    svs = edits[indices]
    # get first sink/source id for the operation id
    sink = svs[0][1]
    source = svs[0][2]
    first_svs = [sink, source]
    # get timestamps
    ts_edit = pd.to_datetime(df.loc[df['operation_id'] == operation_id, 'date'].values[0])
    one_ms = timedelta(milliseconds=1)
    ts_pre_edit = ts_edit - one_ms
    ts_mat = client.materialize.get_timestamp(version=materialization)
    print("pre-edit ts: ", ts_pre_edit)
    print("edit ts:     ", ts_edit)
    print("mat ts:      ", ts_mat)
    # get root ids
    counter = 0
    for sv in first_svs:
        root_id_pre_edit = client.chunkedgraph.get_root_id(sv, ts_pre_edit)
        root_id_edit = client.chunkedgraph.get_root_id(sv, ts_edit)
        root_id_mat =  client.chunkedgraph.get_root_id(sv, ts_mat)
        if counter == 0:
            print("sink sv:          ", sv)
        else:
            print("source sv:        ", sv)
        print("pre-edit root id: ", root_id_pre_edit)
        print("edit root id:     ", root_id_edit)
        print("mat root id:      ", root_id_mat)
        counter += 1

In [17]:
print_root_ids(df, 47222, edits, 943, client)

pre-edit ts:  2020-07-29 09:29:47.045010
edit ts:      2020-07-29 09:29:47.046010
mat ts:       2024-01-22 08:10:01.497934+00:00
sink sv:           97197996596845829
pre-edit root id:  864691135776571232
edit root id:      864691135256230831
mat root id:       864691135256230831
source sv:         97197996596845891
pre-edit root id:  864691135776571232
edit root id:      864691135952048931
mat root id:       864691135952048931


In [18]:
# Was confused why in the above example, the edit root and mat root are the same 
# but it makes sense since it looks the component in question is already a fragment
# and probably ignored subsequently. Checked the component on neuroglancer.
# https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5001713200136192

In [19]:
# Proving that this segment was not touched again
# Checking that the source and sink sv are not in the edit history again after this operation id
print("sink")
edits_for_97197996596845829 = np.where(edits[:, 1:3] == 97197996596845829)
for arr in edits_for_97197996596845829:
    print(edits[arr[0]])
print("source")
edits_for_97197996596845891 = np.where(edits[:, 1:3] == 97197996596845891)
for arr in edits_for_97197996596845891:
    print(edits[arr[0]])

sink
[            47222 97197996596845829 97197996596845891]
[            47222 97197996596845829 97197996596845891]
source
[            47222 97197996596845829 97197996596845891]
[            47222 97197996596845829 97197996596851211]


In [20]:
print_root_ids(df, 105222, edits, 943, client)

pre-edit ts:  2020-08-17 08:09:30.212000
edit ts:      2020-08-17 08:09:30.213000
mat ts:       2024-01-22 08:10:01.497934+00:00
sink sv:           98393028632722115
pre-edit root id:  864691136175092486
edit root id:      864691135502085045
mat root id:       864691135502085045
source sv:         98393028632725107
pre-edit root id:  864691136175092486
edit root id:      864691135850482375
mat root id:       864691135762613558


In [21]:
print_root_ids(df, 304308, edits, 943, client)

pre-edit ts:  2022-01-17 18:59:51.584000
edit ts:      2022-01-17 18:59:51.585000
mat ts:       2024-01-22 08:10:01.497934+00:00
sink sv:           103111169795260140
pre-edit root id:  864691136236982287
edit root id:      864691135785681860
mat root id:       864691135517810826
source sv:         103111169795270503
pre-edit root id:  864691136236982287
edit root id:      864691135785681860
mat root id:       864691135386889089


In [22]:
print_root_ids(df, 1194270, edits, 943, client)

pre-edit ts:  2024-09-28 02:37:20.039000
edit ts:      2024-09-28 02:37:20.040000
mat ts:       2024-01-22 08:10:01.497934+00:00
sink sv:           93388325775407972
pre-edit root id:  864691135939330817
edit root id:      864691135539601266
mat root id:       864691135939330817
source sv:         93388325775409047
pre-edit root id:  864691135939330817
edit root id:      864691135773079883
mat root id:       864691135939330817


In [23]:
# There exists operations that are past the materialization timestamp in the edit history